В 6 модуле мы обучали логистическую регрессию для классификации людей в группу риска ишемической болезни сердца в 10-летней перспрективе по датасету framingham.csv.

Если вы помните, модель получилась плохая: несмотря на довольно большую долю верно классифицированных пациентов (около 85%), она очень плохо определяла пациентов группы риска. Чувствительность была нулевая или почти нулевая, а ошибка 2 рода (ложно-отрицательные результаты среди пациентов группы риска) большая.

Для медицинского теста это плохо, так как врач пропустит много пациентов с высоким риском заболеть.

Проблема заключается вот в чем: в обучающей выборке здоровых пациентов намного больше, чем больных. В этом случае классификатору "выгодно" обучиться так, чтобы хорошо определять бОльший класс, так как он тогда чаще будет угадывать.

Попробуем улучшить работу классификатора двумя способами: настроив веса в логистической регрессии и сделав undersampling здоровых пациентов в обучающей выборке.

In [2]:
# импортируем библиотеки
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
%matplotlib inline

In [3]:
# Импортируем датасет и избавимся от нулевых строк
df = pd.read_csv('framingham.xls')
df.dropna(axis=0,inplace=True) #избавляемся от строчек с пропущенными значениями

# разбиваем датафрейм на две части, dfx - параметры, dfy - целевая переменная. 
dfx = df.drop('TenYearCHD', axis = 1)
dfy = df[['TenYearCHD']] 

# разбиваем датасет на train и test выборку в соотношениии 80% train / 20% test случайным образом
# фиксируем random_state
X_train, X_test, y_train, y_test = train_test_split(dfx, dfy, test_size=0.2, random_state=17) 

1 Способ

Его идея заключается в том, что мы добавим на этапе обучения бОльший штраф за ошибки для более редкого класса, тем самым увеличивая чувствительность классификатора к этому классу. 

Среди параметров LogisticRegression есть class_weight. Он может иметь 3 состояния:

1. class_weight=None означает, что мы обучаем регрессию как обычно, без доп. настроек. Так мы делали в практике 6 модуля 
2. class_weight='balanced' задает веса обратно пропорционально количеству элементов в каждом классе. Например, если в выборке будет 1000 здоровых пациентов и 100 больных, то веса будут относиться как 1:10. В описании параметров на https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression есть формула, по которой считаются коэффициенты.
3. class_weight=dict - можно задать веса самостоятельно с формате словаря.

Для начала давайте попробуем, как работает логистическая регрессия с весами

In [4]:
lm = linear_model.LogisticRegression(solver='liblinear', class_weight='balanced') 
# обучаем
model = lm.fit(X_train, y_train.values.ravel()) 
# сделаем prediction классов на всей тестовой выборке
y_pred = lm.predict(X_test)

In [5]:
# строим confusion matrix - таблицу правильных и неправильных предсказаний
# можно увидеть, что она ведет себя намного лучше, чем для модели без весов!
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[410, 209],
       [ 35,  78]], dtype=int64)

Как видите, мы сущесвтенно улучшили чувствительность классификатора.

Давайте посмотрим на метрики качества:

In [6]:
TN = cnf_matrix[0,0] # True Negative
TP = cnf_matrix[1,1] # True Positive
FN = cnf_matrix[1,0] # False Negative
FP = cnf_matrix[0,1] # False Positive
    
Ac = lm.score(X_test, y_test)
Sens = TP/(TP+FN) 
Sp = TN/(TN+FP)
P = TP/(TP+FP)
typeI = FP/(FP+TN)
typeII = FN/(FN+TP)
    
print('Accuracy: ', Ac)
print('Sensitivity: ', Sens)
print('Specificity: ', Sp)
print('Pricision: ', P)
print('Type I error rate: ', typeI)
print('Type II error rate: ', typeII)

Accuracy:  0.6666666666666666
Sensitivity:  0.6902654867256637
Specificity:  0.6623586429725363
Pricision:  0.27177700348432055
Type I error rate:  0.3376413570274637
Type II error rate:  0.30973451327433627


Accuracy модели стала меньше, чем в простом случае, но зато ошибка второго рода тоже уменьшилась, что для нас в данном случае важнее.

In [7]:
# чтобы не делать 100500 копипастов, создадим функцию print_logit_scores
# которая будет обучать регрессию заданным способом и выводить разные метрики качества

def print_logit_scores(data_train, target_train, data_test, target_test, model_type, weights):
    
    # data_train, target_train, data_test, target_test - это обучающие и тестовые данные
    # model_type задает один из 3 типов обучения: 'n' - None, 'b' - balanced, 'w' - заданные пользователем веса
    # w - вектор весов. Используется только для model_type = 'w'
    
    if (model_type == 'n'): # обучаем с равными весами
        lm = linear_model.LogisticRegression(solver='liblinear', class_weight=None)    
    elif (model_type == 'b'): # балансируем веса, как предлагают разработчики sklearn
        lm = linear_model.LogisticRegression(solver='liblinear', class_weight='balanced')
    elif (model_type == 'w'): # балансируем веса самостоятельно
        lm = linear_model.LogisticRegression(solver='liblinear', class_weight={0:weights[0], 1:weights[1]}) 

    # обучаем
    model = lm.fit(data_train, target_train.values.ravel()) 

    # сделаем prediction классов на всей тестовой выборке
    target_pred = lm.predict(data_test)

    # строим confusion matrix - таблицу правильных и неправильных предсказаний
    cnf_matrix = metrics.confusion_matrix(target_test, target_pred)

    TN = cnf_matrix[0,0] # True Negative
    TP = cnf_matrix[1,1] # True Positive
    FN = cnf_matrix[1,0] # False Negative
    FP = cnf_matrix[0,1] # False Positive
    
    Ac = lm.score(data_test, target_test)
    Sens = TP/(TP+FN) 
    Sp = TN/(TN+FP)
    P = TP/(TP+FP)
    typeI = FP/(FP+TN)
    typeII = FN/(FN+TP)
    
    print('Accuracy: ', Ac)
    print('Sensitivity: ', Sens)
    print('Specificity: ', Sp)
    print('Precision: ', P)
    print('Type I error rate: ', typeI)
    print('Type II error rate: ', typeII)
    
    return [Ac,Sens,Sp,P,typeI,typeII] # возвращаем список метрик

интуитивно хочется поделить веса обратно пропорционально количеству элементов в классе, оставив сумму 1

In [8]:
share = y_train['TenYearCHD'].value_counts()
w0 = share[1]/(share[0]+share[1])
w = np.array([w0,1-w0])
w

array([0.15174299, 0.84825701])

разработчики sklearn предлагают балансировать веса по другому правилу. При этом они тоже будут обратно пропорциональны количествую элементов, но сумма будет уже не 1

In [12]:
np.bincount(y_train['TenYearCHD']) # считает количество вхождений 0 и 1 в y_train['TenYearCHD']
w_b = y_train.shape[0]/ (2*np.bincount(y_train['TenYearCHD']))
w_b

array([0.589444  , 3.29504505])

In [13]:
# Давайте убедимся, что отношения весов действительно одинаковые
# При этом бОльший по размеру класс (нулевой, то есть здоровые пациенты) имеет мЕньший вес
print('отношение интуитивных весов: ', w[0]/w[1])
print('отношение balanced весов: ', w_b[0]/w_b[1])

отношение интуитивных весов:  0.17888799355358584
отношение balanced весов:  0.17888799355358584


In [14]:
# сравним 
print('ручная балансировка по правилу balanced')
m1 = print_logit_scores(X_train, y_train, X_test, y_test, 'w', w_b) # ручная балансировка по правилу balanced
print('\n')
print ('встроенная балансировка по правилу balanced')
m2 = print_logit_scores(X_train, y_train, X_test, y_test, 'b', w) # встроенная балансировка по правилу balanced
print('\n')
print ('без балансировки весов')
m3 = print_logit_scores(X_train, y_train, X_test, y_test, 'n', w) # без балансировки весов

ручная балансировка по правилу balanced
Accuracy:  0.6666666666666666
Sensitivity:  0.6902654867256637
Specificity:  0.6623586429725363
Precision:  0.27177700348432055
Type I error rate:  0.3376413570274637
Type II error rate:  0.30973451327433627


встроенная балансировка по правилу balanced
Accuracy:  0.6666666666666666
Sensitivity:  0.6902654867256637
Specificity:  0.6623586429725363
Precision:  0.27177700348432055
Type I error rate:  0.3376413570274637
Type II error rate:  0.30973451327433627


без балансировки весов
Accuracy:  0.855191256830601
Sensitivity:  0.07964601769911504
Specificity:  0.9967689822294022
Precision:  0.8181818181818182
Type I error rate:  0.0032310177705977385
Type II error rate:  0.9203539823008849


2 Способ

Его идея заключается в том, чтобы уравнять доли "здоровых" и "больных" в обучающей выборке.

Каким образом?

Очень просто: из всех "здоровых" пациентов в обучающей выборке сделам подвыборку того же размера, сколько у нас "больных". Например, если в обучающей выборке 1000 "здоровых" и 100 "больных", то мы из этой 1000 случайным образом выберем 100. Это называется undersampling

In [ ]:
conda install -c conda-forge imbalanced-learn    #попытки установить библиотеку

In [16]:
pip install imblearn --user    #попытки установить библиотеку

  Found existing installation: scikit-learn 0.20.3
    Uninstalling scikit-learn-0.20.3:
      Successfully uninstalled scikit-learn-0.20.3
Note: you may need to restart the kernel to use updated packages.


Could not install packages due to an EnvironmentError: [WinError 5] Отказано в доступе: 'd:\\datascience\\anaconda\\lib\\site-packages\\~klearn\\__check_build\\_check_build.cp37-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [15]:
# Нам понадобится новая библиотека imblearn
# в моей версии Anaconda (2019.03) она не предустановлена
# возможно, вам тоже нужно установить ее самостоятельно
from imblearn.under_sampling import RandomUnderSampler

ModuleNotFoundError: No module named 'imblearn'

In [ ]:
# освежите в памяти, что показывает share и что значит share[1]
# параметр ratio в RandomUnderSampler задается словарем: 
# 1: - желаемое количество объектов класса 1
# 0: - желаемое количество объектов класса 0

# задаем параметры выборки:
sampler = RandomUnderSampler(ratio={1: share[1], 0: share[1]})

# сам unpersampling выполняется здесь:
X_train_under_np, y_train_under_np = sampler.fit_sample(X_train, y_train)

# преобразуем в DataFrame, чтобы скормить логистической регрессии
X_train_under = pd.DataFrame(X_train_under_np)
y_train_under = pd.DataFrame(y_train_under_np)

In [ ]:
# вычисляем качество модели с undersampling
# как вы думаете, почему в качестве model_type здесь можно взять 'n'?
m_u = print_logit_scores(X_train_under, y_train_under, X_test, y_test, 'n', w)

In [ ]:
# сравните результаты со встроенной балансировкой на всей обучающей выборке
# как вы думаете, какой есть сущесвтенный недостаток у undersampling по сравнению с балансировкой весов?
m2 = print_logit_scores(X_train, y_train, X_test, y_test, 'b', w)

Задача 8.4. Условие
Загрузите датасеты Admission_train.csv и Admission_test.csv.

Преобразуйте данные в столбце Chance of Admit в 0 и 1 по правилу: 0, если вероятность меньше 0.5, и 1, если больше либо равна 0.5. Удалите столбцы: Unnamed: 0 и Serial No.

Выполните задания, которые приведены ниже:

Задание 8.4.1

Обучите логистическую регрессию по всем признакам без балансировки весов. Вычислите accuracy, чувствительность, специфичность, точность и ошибки 1 и 2 рода

In [51]:
# Импортируем датасет и избавимся от нулевых строк
df_train = pd.read_csv('Admission_train.csv')
#df_train.dropna(axis=0,inplace=True) #избавляемся от строчек с пропущенными значениями
df_train.drop(['Unnamed: 0'], axis=1, inplace=True)
df_train.drop(['Serial No.'], axis=1, inplace=True)

df_test = pd.read_csv('Admission_test.csv')
#df_test.dropna(axis=0,inplace=True)
df_test.drop(['Unnamed: 0'], axis=1, inplace=True)
df_test.drop(['Serial No.'], axis=1, inplace=True)

df_train.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,318,109,3,3.5,4.0,9.22,1,0.68
1,336,118,5,4.5,4.0,9.19,1,0.92
2,324,110,3,3.5,3.0,9.22,1,0.89
3,334,120,5,4.0,5.0,9.87,1,0.97
4,312,103,3,3.5,4.0,8.78,0,0.67


In [52]:
df_train.columns=['GRE', 'TOEFL', 'University Rating', 'SOP', 'LOR', 'CGPA', 'Research', 'Chance']
df_test.columns=['GRE', 'TOEFL', 'University Rating', 'SOP', 'LOR', 'CGPA', 'Research', 'Chance']
df_test.head()

,GRE,TOEFL,University Rating,SOP,LOR,CGPA,Research,Chance
0,324,111,5,4.5,4.0,9.16,1,0.90
1,314,107,3,3.0,3.5,8.17,1,0.73
2,295,99,1,2.0,1.5,7.57,0,0.37
3,324,111,3,2.5,1.5,8.79,1,0.70
4,297,100,1,1.5,2.0,7.90,0,0.52


In [54]:
def func(x):
    if x<0.5:
        return 0
    else:
        return 1

In [55]:
train=df_train.Chance.apply(func)
df_train.Chance=train

test=df_test.Chance.apply(func)
df_test.Chance=test

df_test.head()

,GRE,TOEFL,University Rating,SOP,LOR,CGPA,Research,Chance
0,324,111,5,4.5,4.0,9.16,1,1
1,314,107,3,3.0,3.5,8.17,1,1
2,295,99,1,2.0,1.5,7.57,0,0
3,324,111,3,2.5,1.5,8.79,1,1
4,297,100,1,1.5,2.0,7.90,0,1


In [61]:
X_train=df_train.drop('Chance', axis = 1)
y_train=df_train[['Chance']]
X_test=df_test.drop('Chance', axis = 1)
y_test=df_test[['Chance']]

In [62]:
lm = linear_model.LogisticRegression(solver='liblinear', class_weight=None) 
# обучаем
model = lm.fit(X_train, y_train.values.ravel()) 
# сделаем prediction классов на всей тестовой выборке
y_pred = lm.predict(X_test)

In [65]:
# строим confusion matrix - таблицу правильных и неправильных предсказаний
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[ 3,  6],
       [ 3, 88]], dtype=int64)

In [66]:
TN = cnf_matrix[0,0] # True Negative
TP = cnf_matrix[1,1] # True Positive
FN = cnf_matrix[1,0] # False Negative
FP = cnf_matrix[0,1] # False Positive
    
Ac = lm.score(X_test, y_test)
Sens = TP/(TP+FN) 
Sp = TN/(TN+FP)
P = TP/(TP+FP)
typeI = FP/(FP+TN)
typeII = FN/(FN+TP)
    
print('Accuracy: ', Ac)
print('Sensitivity: ', Sens)
print('Specificity: ', Sp)
print('Pricision: ', P)
print('Type I error rate: ', typeI)
print('Type II error rate: ', typeII)

Accuracy:  0.91
Sensitivity:  0.967032967032967
Specificity:  0.3333333333333333
Pricision:  0.9361702127659575
Type I error rate:  0.6666666666666666
Type II error rate:  0.03296703296703297


Задание 8.4.2
 
Обучите логистическую регрессию по всем признакам со встроенной балансировкой весов. Вычислите на тестовой выборке Accuracy, чувстительность, специфичность, точность, вероятности ошибок 1 и 2 рода.

Ответы округлите до 4 знаков:

In [67]:
lm = linear_model.LogisticRegression(solver='liblinear', class_weight='balanced') 
# обучаем
model = lm.fit(X_train, y_train.values.ravel()) 
# сделаем prediction классов на всей тестовой выборке
y_pred = lm.predict(X_test)

In [68]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[ 7,  2],
       [20, 71]], dtype=int64)

In [69]:
TN = cnf_matrix[0,0] # True Negative
TP = cnf_matrix[1,1] # True Positive
FN = cnf_matrix[1,0] # False Negative
FP = cnf_matrix[0,1] # False Positive
    
Ac = lm.score(X_test, y_test)
Sens = TP/(TP+FN) 
Sp = TN/(TN+FP)
P = TP/(TP+FP)
typeI = FP/(FP+TN)
typeII = FN/(FN+TP)
    
print('Accuracy: ', Ac)
print('Sensitivity: ', Sens)
print('Specificity: ', Sp)
print('Pricision: ', P)
print('Type I error rate: ', typeI)
print('Type II error rate: ', typeII)

Accuracy:  0.78
Sensitivity:  0.7802197802197802
Specificity:  0.7777777777777778
Pricision:  0.9726027397260274
Type I error rate:  0.2222222222222222
Type II error rate:  0.21978021978021978
